In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

np.random.seed(2223)

The function *Kroupa IMF* generates stellar masses according to Kroupa IMF distribution (2001) in the range [0.01,50] M$_\odot$. It returns *m$_{tot}$* (1-D array), i.e., the masses of all the stars in the system in M$_\odot$.

In [2]:
def Kroupa_IMF(N=1, m_min=0.01, m_max=50):
    
    alpha_1 = 0.3
    alpha_2 = 1.3
    alpha_3 = 2.3
    
    m_change_1 = 0.08
    m_change_2 = 0.5
    
    continuity_1 = m_change_1**(alpha_2-alpha_1)
    continuity_2 = continuity_1*m_change_2**(alpha_3-alpha_2)

    norm_1 = (m_change_1**(1-alpha_1)-m_min**(1-alpha_1))/(1-alpha_1)
    norm_2 = continuity_1*(m_change_2**(1-alpha_2)-m_change_1**(1-alpha_2))/(1-alpha_2)
    norm_3 = continuity_2*(m_max**(1-alpha_3)-m_change_2**(1-alpha_3))/(1-alpha_3)

    norm = norm_1 + norm_2 + norm_3

    P_m = np.random.uniform(0,1,N)
    mask_light_1 = (norm*P_m - norm_1 < 0)
    mask_light_2 = (norm*P_m - norm_2 - norm_1 < 0) & (norm*P_m - norm_1 > 0)
    mask_heavy = (norm*P_m - norm_2 - norm_1 > 0)

    P_light_1 = P_m[mask_light_1]
    P_light_2 = P_m[mask_light_2]
    P_heavy = P_m[mask_heavy]

    m_light_1 = ((1-alpha_1)*(norm*P_light_1) + m_min**(1-alpha_1))**(1/(1-alpha_1))
    m_light_2 = ((1-alpha_2)*(norm*P_light_2-norm_1)/continuity_1 + m_change_1**(1-alpha_2))**(1/(1-alpha_2))
    m_heavy = ((1-alpha_3)*(norm*P_heavy-norm_1-norm_2)/continuity_2 + m_change_2**(1-alpha_3))**(1/(1-alpha_3))

    m_tot = np.concatenate((m_light_1,m_light_2,m_heavy), axis=0)
    np.random.shuffle(m_tot)
    
    return m_tot

The function *selection masses* extracts a sample of stellar masses from the Kroupa IMF within a selected range of values [*m*$_{min}$,*m*$_{max}$] by using a accept-reject method; hence it returns *masses* (1-D array), i.e., the masses of all the sampled stars in M$_\odot$.

In [3]:
def selection_masses(N_masses, m_min, m_max):
     
    masses = Kroupa_IMF(N=N_masses)
    
    while True:
        mask_masses = (masses < m_min) | (masses > m_max)
        masses_1 = masses[~mask_masses]
        N_rejected = mask_masses.sum()
        if N_rejected == 0:
            break
        masses_2 = Kroupa_IMF(N=N_rejected)
        masses = np.concatenate((masses_1,masses_2), axis=0)

    np.random.shuffle(masses)
    
    return masses

The function *assign masses* takes all the extracted stellar masses and assigns them to single stars and binary components; then, it performs a random pairing between binary components to create binary systems, provided that the secondary mass is always lower than the primary one. Thus, the function returns:

1) *m$_s$* : masses of single stars in M$_\odot$ (1-D array);

2) *m$_1$* : masses of primary components in M$_\odot$ (1-D array);

3) *m$_2$* : masses of secondary components in M$_\odot$ (1-D array);

4) *m$_{cm}$* : masses of binary centres of mass in M$_\odot$ (1-D array);

5) *m* : ordered masses of single stars and binary centres of mass in M$_\odot$ (1-D array);

6) *q* : mass ratio (1-D array).

In [4]:
def assign_masses(N_s, N_b, m_tot):
    
    m_s = m_tot[:N_s]
    m_s = m_s.reshape(N_s)
    m_b = m_tot[N_s:]
 
    m_1, m_2, q = np.zeros(N_b), np.zeros(N_b), np.zeros(N_b)
    
    m_b = m_b.reshape(N_b, 2)
    mask_change = (m_b[:,0] - m_b[:,1] < 0)
    m_1[~mask_change], m_2[~mask_change] = m_b[~mask_change][:,0], m_b[~mask_change][:,1]
    m_1[mask_change], m_2[mask_change] = m_b[mask_change][:,1], m_b[mask_change][:,0]
    
    q = m_2/m_1

    m_cm = m_1 + m_2
    m = np.concatenate((m_s,m_cm), axis=0)
    
    return m_s, m_1, m_2, m_cm, m, q

The function *find remnants* distinguish between WDs, NSs and BHs according to the value of the mass; in particular, WD masses are assumed to be in the range [1,1.44[ M$_\odot$, whereas NS ones in the range [1.44,3[ M$_\odot$ and BH ones in the range [3,50] M$_\odot$. As a consequence, a star is classified as dark remnant if its mass is greater than or equal to 1 M$_\odot$. The function, applied to all the stars in the system, returns:

1) *N$_{remn}$* : total number of remnants (int type);

2) *M$_{remn}$* : total remnant mass in M$_\odot$ (1-D array).

In [5]:
def find_remnants(m_tot):
    
    mask_WD = (m_tot >= 1) & (m_tot < 1.44)
    mask_NS = (m_tot >= 1.44) & (m_tot < 3)
    mask_BH = (m_tot >= 3)
    mask_remn = (m_tot >= 1)
    
    m_WD = m_tot[mask_WD]
    m_NS = m_tot[mask_NS]
    m_BH = m_tot[mask_BH]

    M_WD = np.sum(m_WD)
    M_NS = np.sum(m_NS)
    M_BH = np.sum(m_BH)
    M_remn = M_WD + M_NS + M_BH
    
    N_WD = mask_WD.sum()
    N_NS = mask_NS.sum()
    N_BH = mask_BH.sum()
    N_remn = N_WD + N_NS + N_BH
    
    return N_remn, M_remn

The function *count stars* counts the number of stars in a given evolutionary phase depending on their mass value: in particular, MS stars have mass in the range [0.1,0.725[ M$_\odot$, whereas RGB stars in the range [0.725,1[ M$_\odot$ and WDs in the range [1,1.44[ M$_\odot$. Hence the function returns:

1) *N$_{MS}$* : number of MS stars (int type);

2) *N$_{RGB}$* : number of RGB stars (int type);

3) *N$_{WD}$* : number of WD stars (int type).

Since these variables are used to determine the total luminosity of the system when computing the related mass-to-light ratio in the output analysis, the function must be initialized before any cut procedure takes place.

In [ ]:
def count_stars(m_tot):
    
    mask_MS = (m_tot < 0.725)
    mask_RGB = (m_tot >= 0.725) & (m_tot < 1)
    mask_WD = (m_tot >= 1) & (m_tot < 1.44)
    
    N_MS = mask_MS.sum()
    N_RGB = mask_RGB.sum()
    N_WD = mask_WD.sum()
            
    return N_MS, N_RGB, N_WD

The function *position* determines the positions of single stars and binary centres of mass according to the Plummer model, and returns:

1) *r* : position vectors' magnitudes in pc (1-D array);

2) *X* : position vectors' components in pc (*N*$\times$3 matrix, each line referring to a star and each column to a position component, i.e., *x*, *y*, *z*).

In [7]:
def position(N, R):
    
    P_r = np.random.uniform(0,1,N)
    P_theta = np.random.uniform(-1,1,N)
    P_phi = np.random.uniform(0,2*np.pi,N)

    theta = np.arccos(P_theta)
    phi = P_phi
    
    r = (1/np.sqrt(pow(P_r,-2/3) - 1))*R
    
    X = np.zeros(shape=(N,3))
    
    X[:,0] = r*np.sin(theta)*np.cos(phi)
    X[:,1] = r*np.sin(theta)*np.sin(phi)
    X[:,2] = r*np.cos(theta)
    
    return r, X

The function *single position* assigns the extracted positions to single stars; therefore, it returns single stellar position vectors' components in pc (*X$_s$* : *N$_s$*$\times$3 matrix, each line referring to a star and each column to a position component, i.e., *x$_s$*, *y$_s$*, *z$_s$*).

In [8]:
def single_position(N_s, r, X):
    
    r_s = r[:N_s]
    X_s = X[:N_s]
    
    return X_s

The function *binary position* determines the positions of both binary centres of mass and binary components, by assuming as a location the apocentre of their respective orbit for computational convenience. In particular, binary components' positions are evaluated in the system reference frame. The function returns:

1) *X$_{cm}$* : position vectors' components of binary centres of mass in pc (*N$_b$*$\times$3 matrix, each line referring to a binary system and each column to a position component, i.e., *x$_{cm}$*, *y$_{cm}$*, *z$_{cm}$*);

2) *X$_1$* : position vectors' components of primaries in pc (*N$_b$*$\times$3 matrix, each line referring to a primary star and each column to a position component, i.e., *x$_1$*, *y$_1$*, *z$_1$*);

2) *X$_2$* : position vectors' components of secondaries in pc (*N$_b$*$\times$3 matrix, each line referring to a secondary star and each column to a position component, i.e., *x$_2$*, *y$_2$*, *z$_2$*).

In [9]:
def binary_position(N_s, N_b, m_1, m_2, r, X, a, e):

    r_apo = a*(1+e)

    nx = 2*np.random.uniform(0,1,N_b) - 1
    ny = 2*np.random.uniform(0,1,N_b) - 1
    nz = 2*np.random.uniform(0,1,N_b) - 1 
    
    nx_norm = nx/np.sqrt(nx**2 + ny**2 + nz**2)
    ny_norm = ny/np.sqrt(nx**2 + ny**2 + nz**2)
    nz_norm = nz/np.sqrt(nx**2 + ny**2 + nz**2)
    
    X_apo = np.zeros(shape=(N_b,3))
    X_apo[:,0] = nx_norm*r_apo
    X_apo[:,1] = ny_norm*r_apo
    X_apo[:,2] = nz_norm*r_apo
    
    # Position of binary centres of mass
    
    r_cm = r[N_s:]
    X_cm = X[N_s:]
    
    # Position of primary components
    
    X_1 = np.zeros(shape=(N_b,3))
    
    X_1[:,0] = X_cm[:,0] + m_2*X_apo[:,0]/(m_1+m_2)
    X_1[:,1] = X_cm[:,1] + m_2*X_apo[:,1]/(m_1+m_2)
    X_1[:,2] = X_cm[:,2] + m_2*X_apo[:,2]/(m_1+m_2)
    
    r_1 = np.sqrt(X_1[:,0]**2 + X_1[:,1]**2 + X_1[:,2]**2)

    # Position of secondary components
    
    X_2 = np.zeros(shape=(N_b,3))
    
    X_2[:,0] = X_cm[:,0] - m_1*X_apo[:,0]/(m_1+m_2)
    X_2[:,1] = X_cm[:,1] - m_1*X_apo[:,1]/(m_1+m_2)
    X_2[:,2] = X_cm[:,2] - m_1*X_apo[:,2]/(m_1+m_2)
    
    r_2 = np.sqrt(X_2[:,0]**2 + X_2[:,1]**2 + X_2[:,2]**2)

    return X_cm, X_1, X_2

The function *velocity* extracts the velocities of both single stars and binary centres of mass according to the Plummer model, provided that they do not exceed in magnitude the associated escape velocity (condition evaluated through an accept-reject method). Thus, it returns:

1) *v* : velocity vectors' magnitudes in km/s (1-D array);

2) *V* : velocity vectors' components in km/s (*N*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_x$*, *v$_y$*, *v$_z$*).

In [10]:
def velocity(N, M_tot, R, r):
    
    n_samples = 0
    v = []

    while n_samples < N:
        x_t = np.random.uniform(0,1)
        y_t = 0.1*np.random.uniform(0,1)
        
        if (x_t**2)*(1-x_t**2)**(3.5) > y_t: 
            v.append(x_t)
            n_samples +=1
            
    r = r/R
    ve = np.sqrt(2)*(1+r**2)**(-0.25)
    
    v = np.asarray(v)*ve*np.sqrt(G*M_tot/R) 
        
    n1 = np.random.uniform(0,1,N)
    n2 = np.random.uniform(0,1,N)

    V = np.zeros(shape=(N,3))

    V[:,0] = (1 - 2*n1)*v
    V[:,1] = np.sqrt(pow(v,2) - pow(V[:,0],2))*np.sin(2*np.pi*n2)
    V[:,2] = np.sqrt(pow(v,2) - pow(V[:,0],2))*np.cos(2*np.pi*n2)

    return v, V

The function *virial ratio* calculates the kinetic and the potential energy of the system (indicated by *T* and *U*, respectively), which are used to determine the virial ratio *Q* (float type). The purpose of the function is checking that the virial equilibrium is effectively obtained, i.e., that *Q* = 0.5.

In [11]:
def virial_ratio(G, m, X, v):

    T = np.sum(m*pow(v,2)/2)
    
    U = 0
    for i in range(len(X)-1):
        x_eff = X[i+1:,0]
        y_eff = X[i+1:,1]
        z_eff = X[i+1:,2]
        l = len(x_eff)
        
        x_i = X[i,0]*np.ones(l)
        y_i = X[i,1]*np.ones(l) 
        z_i = X[i,2]*np.ones(l)
        
        inverse_distance = 1/np.sqrt((x_i - x_eff)**2 + (y_i - y_eff)**2 +(z_i-z_eff)**2)
        U += -G*m[i]*np.sum(np.dot(m[i+1:],inverse_distance))
                
    Q = T/abs(U)
    
    print(Q)
    
    return Q

The function *single velocity* assigns the extracted velocities to single stars and returns single stellar velocity vectors' components in km/s (*V$_s$* : *N$_s$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{s,x}$*, *v$_{s,y}$*, *v$_{s,z}$*).

In [12]:
def single_velocity(N_s, v, V):
    
    v_s = v[:N_s]
    V_s = V[:N_s]
    
    return V_s  

The function *binary velocity* assigns the remaining extracted velocities to binary centres of mass and determines the components' velocities by accounting for the additional orbital motion, expressed through the mean orbital velocity. The function returns:

1) *V$_{cm}$* : velocity vectors' components of binary centres of mass in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{cm,x}$*, *v$_{cm,y}$*, *v$_{cm,z}$*);

2) *V$_1$* : velocity vectors' components of primaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{1,x}$*, *v$_{1,y}$*, *v$_{1,z}$*);

3) *V$_2$* : velocity vectors' components of secondaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{2,x}$*, *v$_{2,y}$*, *v$_{2,z}$*).

In [13]:
def binary_velocity(N_s, N_b, m_1, m_2, v, V, a, e):
    
    v_orb_mean = np.sqrt(G*(m_1+m_2)/a) 
    
    nx = 2*np.random.uniform(0,1,N_b) - 1
    ny = 2*np.random.uniform(0,1,N_b) - 1
    nz = 2*np.random.uniform(0,1,N_b) - 1

    nx_norm = nx/np.sqrt(nx**2 + ny**2 + nz**2)
    ny_norm = ny/np.sqrt(nx**2 + ny**2 + nz**2)
    nz_norm = nz/np.sqrt(nx**2 + ny**2+ nz**2)
    
    V_orb_mean = np.zeros(shape=(N_b,3))
    
    V_orb_mean[:,0] = nx_norm*v_orb_mean
    V_orb_mean[:,1] = ny_norm*v_orb_mean
    V_orb_mean[:,2] = nz_norm*v_orb_mean
    
    # Velocity of binary centres of mass

    v_cm = v[N_s:]
    V_cm = V[N_s:]
    
    # Velocity of primary stars
    
    V_1 = np.zeros(shape=(N_b,3))

    V_1[:,0] = V_cm[:,0] + m_2*V_orb_mean[:,0]/(m_1+m_2)
    V_1[:,1] = V_cm[:,1] + m_2*V_orb_mean[:,1]/(m_1+m_2)
    V_1[:,2] = V_cm[:,2] + m_2*V_orb_mean[:,2]/(m_1+m_2)
    
    v_1 = np.sqrt(V_1[:,0]**2 + V_1[:,1]**2 + V_1[:,2]**2)
    
    # Velocity of secondary stars
    
    V_2 = np.zeros(shape=(N_b,3))
    
    V_2[:,0] = V_cm[:,0] - m_1*V_orb_mean[:,0]/(m_1+m_2)
    V_2[:,1] = V_cm[:,1] - m_1*V_orb_mean[:,1]/(m_1+m_2)
    V_2[:,2] = V_cm[:,2] - m_1*V_orb_mean[:,2]/(m_1+m_2)
 
    v_2 = np.sqrt(V_2[:,0]**2 + V_2[:,1]**2 + V_2[:,2]**2)
  
    return V_cm, V_1, V_2

The function *semi-major axis* extracts binary semi-major axes from a logarithmically flat distribution in the desired range of values [*a$_{min}$*,*a$_{max}$*] (AU), and returns:

1) *a$_{AU}$* : semi-major axes in AU (1-D array);

2) *a* : semi-major axes in pc (1-D array).

In [14]:
def semi_major_axis(N_b, a_au_min, a_au_max):
    
    X_a = np.random.uniform(0,1,N_b)
    norm_a = np.log(a_au_max/a_au_min)
     
    a_au = np.exp((norm_a*X_a)+np.log(a_au_min))
    a = a_au/206264.55529277
    
    return a_au, a

The function *orbital period* calculates the orbital period of each binary system by using Kepler's third law; it returns the orbital period *P* in days (1-D array).

In [15]:
def period(m_1, a_au, q, G_au=39.478):
    
    P_yrs = np.sqrt((4*np.pi**2*a_au**3)/(G_au*m_1*(1+q))) # yrs
    P = P_yrs*365 # days
    
    return P

The function *eccentricity* samples eccentricities from a thermal distribution in the desired range of values [*e*$_{min}$,*e*$_{max}$], hence returning the eccentricity *e* (1-D array).

In [16]:
def eccentricity(N_b, e_min, e_max):

    norm_e = pow(e_max,2) - pow(e_min,2)
    X_e = np.random.uniform(0,1,N_b)
    e = np.sqrt(norm_e*X_e + pow(e_min,2))
  
    return e

The function *isochrone parameters* reads and processes the data of the isochrone relative to the age 13 Gyr. In particular, it returns:

1) *M$_{MS}$* : masses of MS stars in M$_\odot$ (1-D array);

2) *M$_{RGB}$* : masses of RGB stars in M$_\odot$ (1-D array);

3) *$\log{(L_{MS})}$* : logarithm of MS luminosities (1-D array);

4) *$\log{(L_{RGB})}$* : logarithm of RGB luminosities (1-D array).

In [17]:
def isochrone_parameters():
    
    data = []
    with open('Isochrone','r') as file:
        for line in file:
        data.append([float(x) for x in line.split()])

    data = np.asarray(data)

    RGB_iso = data[data[:,9] == 3] # Label 3 defines RGB stars
    MS_iso = data[data[:,9] < 3] # Labels 1 and 2 define MS stars

    M_MS, log_L_MS = MS_iso[:,5], MS_iso[:,6]
    M_RGB, log_L_RGB = RGB_iso[:,5], RGB_iso[:,6]

    order_MS = np.argsort(M_MS)
    M_MS = M_MS[order_MS]
    log_L_MS = log_L_MS[order_MS]

    order_RGB = np.argsort(M_RGB)
    M_RGB = M_RGB[order_RGB]
    log_L_RGB = log_L_RGB[order_RGB]
                            
    return  M_MS, M_RGB, log_L_MS, log_L_RGB

The following block of functions is intended to perform a linear interpolation of the isochrone, with the ultimate purpose of attributing a luminosity and temperature to each mass value. Specifically, the function *find your place* finds the isochrone mass nearest to the extracted one, which is then used by the function *linear interpolation* to predict the associated luminosity and temperature; finally, the functions *MS data* and *RGB data* calculate the luminosity and temperature of MS and RGB stars, respectively, starting from the predicted values. Thus, the most important returned variables of the block are:

1) *L$_{MS}$* : luminosities of MS stars in L$_\odot$ (1-D array);

2) *T$_{MS}$* : temperatures of MS stars in K (1-D array);

3) *L$_{RGB}$* : luminosities of RGB stars in L$_\odot$ (1-D array);

4) *T$_{RGB}$* : temperatures of RGB stars in K (1-D array).

In [18]:
# Isochrone fit

def find_your_place(mass, M_iso): # It finds the index of nearest experimental mass value to the selected mass value of the isochrone
    
    where = np.argmin(np.abs(M_iso-mass))

    if -(M_iso[where]-mass) > 0:
        return where
    else: 
        return where -1


def linear_interpolation(M_iso, log_L_iso, log_T_iso, mass):
    
    where = find_your_place(mass, M_iso)

    predicted_log_L = log_L_iso[where]+(log_L_iso[where+1]-log_L_iso[where])/(M_iso[where+1]-M_iso[where])*(mass-M_iso[where])
    predicted_log_T = log_T_iso[where]+(log_T_iso[where+1]-log_T_iso[where])/(M_iso[where+1]-M_iso[where])*(mass-M_iso[where])
    
    return predicted_log_L, predicted_log_T


def RGB_data(M_RGB, log_L_RGB, log_T_RGB, mass):
    
    if mass < 0.785:
        pred_log_L_RGB, pred_log_T_RGB = linear_interpolation(M_RGB, log_L_RGB, log_T_RGB, mass)
    else:
        where = find_your_place(mass, M_RGB)
        pred_log_L_RGB = log_L_RGB[where]
        pred_log_T_RGB = log_T_RGB[where]
        
    L_RGB = np.exp(pred_log_L_RGB)
    T_RGB = np.exp(pred_log_T_RGB)
    
    return L_RGB, T_RGB


def MS_data(M_MS, log_L_MS, log_T_MS, mass):
    
    pred_log_L_MS, pred_log_T_MS = linear_interpolation(M_MS, log_L_MS, log_T_MS, mass)
    L_MS = np.exp(pred_log_L_MS)
    T_MS = np.exp(pred_log_T_MS)
    
    return L_MS, T_MS

The function *HRD data* assigns a luminosity and temperature to each star in the system depending on its mass value; in particular, MS and RGB stars are given a luminosity value derived from the isochrone linear interpolation, whereas dark remnants a fixed one (10$^{-4}$ as for WDs, while 0 as for both NSs and BHs). The function returns:

1) *L$_{star}$* : luminosities associated to a certain set of stellar masses in units of L$_\odot$ (1-D array);

2) *T$_{star}$* : temperatures relative to a certain set of stellar masses in units of K (1-D array).

In [ ]:
def HRD_data(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, masses):
    
    L_star = []
    T_star = []
    
    for mass in masses:
        if mass < 0.725:
            L_MS, T_MS = MS_data(M_MS, log_L_MS, log_T_MS, mass)
            L_star.append(L_MS)
            T_star.append(T_MS)
        elif mass < 1:
            L_RGB, T_RGB = RGB_data(M_RGB, log_L_RGB, log_T_RGB, mass)
            L_star.append(L_RGB)
            T_star.append(T_RGB)
        elif mass < 1.44:
            L_star.append(10**(-4))
            T_star.append(10**6)
        elif mass >= 1.44:
            L_star.append(0)
            T_star.append(10**6)
            
    L_star = np.asarray(L_star)
    T_star = np.asarray(T_star)
    
    return L_star, T_star

The function *luminosity temperature* applies the function *HRD data* to both single stars and binary components, returning:

1) *L$_s$* : luminosities of single stars in L$_\odot$ (1-D array);

2) *L$_1$* : luminosities of primary stars in L$_\odot$ (1-D array);

3) *L$_2$* : luminosities of secondary stars in L$_\odot$ (1-D array);

4) *T$_s$* : temperatures of single stars in K (1-D array);

5) *T$_1$* : temperatures of primary stars in K (1-D array);

6) *T$_2$* : temperatures of secondary stars in K (1-D array).

In [ ]:
def luminosity_temperature(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, m_s, m_1, m_2):
    
    L_s, T_s = HRD_data(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, m_s)
    L_1, T_1 = HRD_data(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, m_1)
    L_2, T_2 = HRD_data(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, m_2)
    
    return L_s, T_s, L_1, L_2, T_1, T_2

The function *stellar radii* calculates the photospheric radii of binary components by using the luminosity-temperature relation; in particular, luminosities are converted from L$_\odot$ to J, whereas stellar radii from m to pc. The function returns:

1) *R$_1$* : photospheric radii of primary stars in pc (1-D array);

2) *R$_2$* : photospheric radii of secondary stars in pc (1-D array).

In [ ]:
def stellar_radii(L_1, L_2, T_1, T_2):
    
    h_bar = 1.0551*10**(-34)
    k_B = 1.380649*10**(-23) # J/K
    c = 299792458 # km/s
    
    sigma = (np.pi**2*k_B**4)/(60*h_bar**3*c**2) # Stefan-Boltzmann constant
    
    R_1 = 3.2407*10**(-17)*(L_1*3.827*10**(26)/(4*np.pi*sigma*T_1**4))**(1/2) # from m to pc
    
    R_2 = 3.2407*10**(-17)*(L_2*3.827*10**(26)/(4*np.pi*sigma*T_2**4))**(1/2) # from m to pc
    
    return R_1, R_2

The function *luminosity cut-off* applies a cut-off at the TO level to both the single and binary stellar populations: specifically, single stars are rejected if their luminosity is below the TO level, whereas binaries if the sum of their components' luminosities is.
The function returns the following quantities, computed after the luminosity cut-off:

1) *N$_s$* : number of accepted single stars (int type);

2) *N$_b$* : number of accepted binaries (int type);

3) *m$_s$* : masses of accepted single stars in M$_\odot$ (1-D array);

4) *m$_1$* : masses of accepted primaries in M$_\odot$ (1-D array);

5) *m$_2$* : masses of accepted secondaries in M$_\odot$ (1-D array);

6) *V$_s$* : velocity vectors' components of accepted single stars in km/s (*N$_s$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{s,x}$*, *v$_{s,y}$*, *v$_{s,z}$*);

7) *V$_{cm}$* : velocity vectors' components of accepted binary centres of mass in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{cm,x}$*, *v$_{cm,y}$*, *v$_{cm,z}$*);

8) *V$_1$* : velocity vectors' components of accepted primaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{1,x}$*, *v$_{1,y}$*, *v$_{1,z}$*);

9) *V$_2$* : velocity vectors' components of accepted secondaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{2,x}$*, *v$_{2,y}$*, *v$_{2,z}$*);

10) *L$_s$* : luminosities of accepted single stars in L$_\odot$ (1-D array);

11) *L$_1$* : luminosities of accepted primaries in L$_\odot$ (1-D array);

12) *L$_2$* : luminosities of accepted secondaries in L$_\odot$ (1-D array);

13) *R$_1$* : photospheric radii of accepted primaries in pc (1-D array);

14) *R$_2$* : photospheric radii of accepted secondaries in pc (1-D array);

15) *a$_{AU}$* : semi-major axes of accepted binaries in AU (1-D array);

16) *e* : eccentricities of accepted binaries (1-D array).

Note that returned variables are named in the same way as input ones in order for these to be overwritten only in the event that the function is called: otherwise, the code proceeds by using the original variables.

In [ ]:
def luminosity_cut_off(m_s, m_1, m_2, V_s, V_cm, V_1, V_2, L_s, L_1, L_2, R_1, R_2, a, e):
    
    log_L_TO = 0.3
    L_TO = np.exp(log_L_TO)
    
    mask_L_s = (L_s >= L_TO) # condition for a single star to be accepted
    mask_L_b = (L_1 + L_2 >= L_TO) # condition for a binary to be accepted
    
    N_s = mask_L_s.sum()
    N_b = mask_L_b.sum()
    
    m_s = m_s[mask_L_s]
    L_s = L_s[mask_L_s]
    V_s = V_s[mask_L_s]
    
    m_1 = m_1[mask_L_b]
    m_2 = m_2[mask_L_b]
    V_cm = V_cm[mask_L_b]
    V_1 = V_1[mask_L_b]
    V_2 = V_2[mask_L_b]
    L_1 = L_1[mask_L_b]
    L_2 = L_2[mask_L_b]
    R_1 = R_1[mask_L_b]
    R_2 = R_2[mask_L_b]
    a = a[mask_L_b]
    e = e[mask_L_b]
    
    return N_s, N_b, m_s, m_1, m_2, V_s, V_cm, V_1, V_2, L_s, L_1, L_2, R_1, R_2, a, e, f_b

The function *RLOF* evaluates the occurrence of the RLOF phenomenon between close binary components through the modified Eggleton's formula, and distinguishes between accepted and rejected binaries: the former maintain their individual stellar identity, whereas the latter are treated as mergers, namely as an unique object moving at the center-of-mass speed and having luminosity equal to the sum of the components' luminosities. A less conservative version of the RLOF criterion consists in considering uncertain cases, where only primaries exceed the respective Roche lobe, instead of mergers: by selecting the # variable *mask_RLOF*, then, the effects of this alternative option can be explored.
The function returns:

1) *N$_{b,acc}$* : number of accepted binaries (int type);

2) *N$_{b,rej}$* : number of rejected binaries (int type);

3) *V$_{1,acc}$* : velocity vectors' components of accepted primaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{1,acc_x}$*, *v$_{1,acc_y}$*, *v$_{1,acc_z}$*);

4) *V$_{2,acc}$* : velocity vectors' components of accepted secondaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{2,acc_x}$*, *v$_{2,acc_y}$*, *v$_{2,acc_z}$*);

5) *V$_{cm,rej}$* : velocity vectors' components of rejected binaries in km/s (*N$_b$*$\times$3 matrix, each line referring to a star and each column to a velocity component, i.e., *v$_{cm,rej_x}$*, *v$_{cm,rej_y}$*, *v$_{cm,rej_z}$*);

6) *L$_{1,acc}$* : luminosities of accepted primaries in L$_\odot$ (1-D array);

7) *L$_{2,acc}$* : luminosities of accepted secondaries in L$_\odot$ (1-D array);

8) *L$_{b,rej}$* : luminosities of rejected binaries in L$_\odot$ (1-D array);

9) *a$_{AU,acc}$* : semi-major axes of accepted binaries in AU (1-D array);

10) *e$_{acc}$* : eccentricities of accepted binaries (1-D array).

In [20]:
def RLOF(N_b, m_s, m_1, m_2, V_1, V_2, V_cm, L_1, L_2, R_1, R_2, a, a_au, e):
    
    # Modified Eggleton's formula
    
    q_1 = m_1/m_2
    q_2 = m_2/m_1
    r_p = a*(1-e) # pericenter radius
    
    R_L_1 = (0.49*q_1**(2/3)*r_p)/(0.6*q_1**(2/3)+np.log(1+q_1**(1/3)))
    R_L_2 = (0.49*q_2**(2/3)*r_p)/(0.6*q_2**(2/3)+np.log(1+q_2**(1/3)))
    
    mask_RLOF = (R_1 > R_L_1) & (R_2 > R_L_2) # merger: both components exceed their Roche lobe
    
    #mask_RLOF = (R_1 > R_L_1) & (R_2 < R_L_2) # uncertain outcome: only primaries exceed their Roche lobe
    
    # Accepted stars' data
    
    N_b_acc = N_b - mask_RLOF.sum()
    V_1_acc = V_1[~mask_RLOF]
    V_2_acc = V_2[~mask_RLOF]
    V_cm_acc = V_cm[~mask_RLOF]
    L_1_acc = L_1[~mask_RLOF]
    L_2_acc = L_2[~mask_RLOF]
    a_au_acc = a_au[~mask_RLOF]
    e_acc = e[~mask_RLOF]
    
    # Rejected stars' data
    
    N_b_rej = mask_RLOF.sum() # Number of mergers
    V_cm_rej = V_cm[mask_RLOF]
    L_b_rej = L_1[mask_RLOF] + L_2[mask_RLOF]
    
    L_b = np.concatenate((L_1_acc,L_2_acc,L_b_rej), axis=0)
            
    return N_b_acc, N_b_rej, V_1_acc, V_2_acc, V_cm_acc, V_cm_rej, L_b, L_b_rej, a_au_acc, e_acc

The function *histograms* displays histograms (if *_plot=True*) that compare:

1) the semi-major axis distribution before and after RLOF rejection;

2) the eccentricity distribution before and after RLOF rejection;

3) the squared velocity distribution of single and binary stars below 5 km/s.

In [21]:
def histograms(V_s, V_1, V_2, a_au, a_au_acc, e, e_acc, _plot=True):

    v_1 = np.sqrt(V_1[:,0]**2 + V_1[:,1]**2 + V_1[:,2]**2)
    v_2 = np.sqrt(V_2[:,0]**2 + V_2[:,1]**2 + V_2[:,2]**2)
    v_s = np.sqrt(V_s[:,0]**2 + V_s[:,1]**2 + V_s[:,2]**2)
    v_b = np.concatenate((v_1,v_2), axis=0)
   
    mask_b = (v_b < 5)
    v_b_cut = v_b[mask_b]
    
    if _plot:
        
        # Semi-major axis distribution before RLOF rejection
        
        plt.figure(figsize=(5,8), dpi=120)
        counts_a, bins_a = np.histogram(a_au, density=True)
        plt.hist(bins_a[:-1], bins_a, weights=counts_a, alpha=0.5, color='silver', edgecolor='black', label='no RLOF')
        plt.xlabel('a [AU]', fontsize=25)
        plt.ylabel('Normalized frequency', fontsize=25)
        plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)
        plt.legend(fontsize=17)
        plt.show()
        
        # Semi-major axis distribution after RLOF rejection
        
        plt.figure(figsize=(5,8), dpi=120)
        counts_a, bins_a = np.histogram(a_au_acc, density=True)   
        plt.hist(bins_a[:-1], bins_a, weights=counts_a, alpha=0.5, color='silver', edgecolor='black', label='RLOF')   
        plt.xlabel('a [AU]', fontsize=25)
        plt.ylabel('Normalized frequency', fontsize=25)
        plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)
        plt.legend(fontsize=17)
        plt.show()
        
        # Eccentricity distribution before RLOF rejection
    
        plt.figure(figsize=(5,8), dpi=120)
        counts_e, bins_e = np.histogram(e, density=True)
        plt.hist(bins_e[:-1], bins_e, weights=counts_e, alpha=0.5, color='silver', edgecolor='black', label='no RLOF')
        plt.xlabel('e', fontsize=25)
        plt.ylabel('Normalized frequency', fontsize=25)
        plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)
        plt.legend(fontsize=17)
        plt.show()
        
        # Eccentricity distribution after RLOF rejection
    
        plt.figure(figsize=(5,8), dpi=120)
        counts_e, bins_e = np.histogram(e_acc, density=True)
        plt.hist(bins_e[:-1], bins_e, weights=counts_e, alpha=0.5, color='silver', edgecolor='black', label='RLOF')
        plt.xlabel('e', fontsize=25)
        plt.ylabel('Normalized frequency', fontsize=25)
        plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)
        plt.legend(fontsize=17)
        plt.show()
        
        # Squared velocity distribution
        
        plt.figure(figsize=(7,8), dpi=120)
        counts_s, bins_s = np.histogram(v_s**2, density=True)
        counts_b, bins_b = np.histogram(v_b_cut**2, density=True)
        plt.hist(bins_s[:-1], bins_s, weights=counts_s, alpha=1, color='silver', label='single stars')
        plt.hist(bins_b[:-1], bins_b, weights=counts_b, alpha=1, histtype='step', color='black', label='binary stars')
        plt.xlabel('$v^2 [km^2/s^2]$', fontsize=25)
        plt.ylabel('Normalized frequency', fontsize=25)
        plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)        
        plt.legend(fontsize=17)
        plt.show()
    
        return 0

The function *velocity dispersion 1* calculates the system velocity dispersion by considering accepted binaries as if they were unresolved (thus including the orbital motion of their respective components), and rejected binaries as represented by their centre of mass velocities.
The returned velocity dispersion value in km/s is *$\sigma_{tot}$* (float type).

In [22]:
def velocity_dispersion_1(N_s, N_b_acc, N_b_rej, V_s, V_cm_rej, V_1_acc, V_2_acc):
    
    N = N_s + 2*N_b_acc + N_b_rej
    
    V = np.zeros(shape=(N,3))
    V[:N_s] = V_s
    V[N_s:N_s+N_b_acc] = V_1_acc
    V[N_s+N_b_acc:N_s+2*N_b_acc] = V_2_acc
    V[N_s+2*N_b_acc:] = V_cm_rej
    
    V_mean = np.zeros(3)
    V_mean[0] = np.mean(V[:,0])
    V_mean[1] = np.mean(V[:,1])
    V_mean[2] = np.mean(V[:,2])
    
    sigma_tot_squared = np.zeros(3)
    sigma_tot_squared[0] = np.sum((V[:,0]-V_mean[0])**2)/N
    sigma_tot_squared[1] = np.sum((V[:,1]-V_mean[1])**2)/N
    sigma_tot_squared[2] = np.sum((V[:,2]-V_mean[2])**2)/N
    
    sigma_tot = np.sqrt(sigma_tot_squared[0] + sigma_tot_squared[1] + sigma_tot_squared[2])
    
    return sigma_tot

The function *velocity dispersion 2* calculates the system velocity dispersion by accounting for the velocity of single stars and binary centres of mass, so that the orbital motion is ruled out: as a consequence, no distinction between accepted and rejected binaries is necessary.
The returned velocity dispersion value in km/s is *$\sigma_{sb}$* (float type).

In [23]:
def velocity_dispersion_2(N_s, N_b_acc, N_b_rej, V_s, V_cm_acc, V_cm_rej):
    
    N = N_s + N_b_acc + N_b_rej
    
    V = np.zeros(shape=(N,3))
    V[:N_s] = V_s
    V[N_s:N_s+N_b_acc] = V_cm_acc
    V[N_s+N_b_acc:] = V_cm_rej
    
    V_mean = np.zeros(3)
    V_mean[0] = np.mean(V[:,0])
    V_mean[1] = np.mean(V[:,1])
    V_mean[2] = np.mean(V[:,2])
    
    sigma_sb_squared = np.zeros(3)
    sigma_sb_squared[0] = np.sum((V[:,0]-V_mean[0])**2)/N
    sigma_sb_squared[1] = np.sum((V[:,1]-V_mean[1])**2)/N
    sigma_sb_squared[2] = np.sum((V[:,2]-V_mean[2])**2)/N
    
    sigma_sb = np.sqrt(sigma_sb_squared[0] + sigma_sb_squared[1] + sigma_sb_squared[2])
    
    return sigma_sb

The function *velocity dispersion 3* evaluates the contribution of single stars, only, to the system velocity dispersion, hence returning the variable *$\sigma_s$* (float type) in km/s.

In [24]:
def velocity_dispersion_3(N_s, V_s):
    
    V_s_mean = np.zeros(3)
    V_s_mean[0] = np.mean(V_s[:,0])
    V_s_mean[1] = np.mean(V_s[:,1])
    V_s_mean[2] = np.mean(V_s[:,2])
    
    sigma_s_squared = np.zeros(3)
    sigma_s_squared[0] = np.sum((V_s[:,0]-V_s_mean[0])**2)/N_s
    sigma_s_squared[1] = np.sum((V_s[:,1]-V_s_mean[1])**2)/N_s
    sigma_s_squared[2] = np.sum((V_s[:,2]-V_s_mean[2])**2)/N_s
    
    sigma_s = np.sqrt(sigma_s_squared[0] + sigma_s_squared[1] + sigma_s_squared[2])
       
    return sigma_s

The function *velocity dispersion 4* calculates the system velocity dispersion by accounting for accepted and rejected binaries as in *velocity dispersion 1*, but weights stellar velocities by the associated luminosities (i.e., accepted binaries by their respective components' luminosities, wheareas rejected binaries by the sum of their components' luminosities).
The returned luminosity-averaged velocity dispersion value in km/s is *$\sigma_{tot,lum}$* (float type).

In [25]:
def velocity_dispersion_4(N_s, N_b_acc, N_b_rej, V_s, V_1_acc, V_2_acc, V_cm_rej, L_s, L_1_acc, L_2_acc, L_b_rej):
    
    N = N_s + 2*N_b_acc + N_b_rej
    
    L_b = np.concatenate((L_1_acc,L_2_acc,L_b_rej), axis=0)
    L = np.concatenate((L_s,L_b), axis=0)
    L_tot = np.sum(L)
    
    V = np.zeros(shape=(N,3))
    V[:N_s] = V_s
    V[N_s:N_s+N_b_acc] = V_1_acc
    V[N_s+N_b_acc:N_s+2*N_b_acc] = V_2_acc
    V[N_s+2*N_b_acc:] = V_cm_rej
    
    V_mean = np.zeros(3)
    V_mean[0] = np.sum(np.dot(L,V[:,0]))/L_tot
    V_mean[1] = np.sum(np.dot(L,V[:,1]))/L_tot
    V_mean[2] = np.sum(np.dot(L,V[:,2]))/L_tot
    
    sigma_tot_lum_squared = np.zeros(3)
    sigma_tot_lum_squared[0] = np.sum(np.dot(L,(V[:,0]-V_mean[0])**2))/L_tot
    sigma_tot_lum_squared[1] = np.sum(np.dot(L,(V[:,1]-V_mean[1])**2))/L_tot                                 
    sigma_tot_lum_squared[2] = np.sum(np.dot(L,(V[:,2]-V_mean[2])**2))/L_tot
                                      
    sigma_tot_lum = np.sqrt(sigma_tot_lum_squared[0] + sigma_tot_lum_squared[1] + sigma_tot_lum_squared[2])                                  
    
    return sigma_tot_lum   

The function *velocity dispersion 5* evaluates the contribution of single stars, only, to the system velocity dispersion, as well as *velocity dispersion 3*, but weights single stellar velocities by the associated luminosities. The return of the function is, therefore, *$\sigma_{s,lum}$* (float type), i.e., the value of the luminosity-averaged velocity dispersion of single stars in km/s.

In [26]:
def velocity_dispersion_5(V_s, L_s):
 
    L_tot = np.sum(L_s)
    
    V_s_mean = np.zeros(3)
    V_s_mean[0] = np.sum(np.dot(L_s,V_s[:,0]))/L_tot
    V_s_mean[1] = np.sum(np.dot(L_s,V_s[:,1]))/L_tot
    V_s_mean[2] = np.sum(np.dot(L_s,V_s[:,2]))/L_tot
    
    sigma_s_lum_squared = np.zeros(3)
    sigma_s_lum_squared[0] = np.sum(np.dot(L_s,(V_s[:,0]-V_s_mean[0])**2))/L_tot
    sigma_s_lum_squared[1] = np.sum(np.dot(L_s,(V_s[:,1]-V_s_mean[1])**2))/L_tot                                 
    sigma_s_lum_squared[2] = np.sum(np.dot(L_s,(V_s[:,2]-V_s_mean[2])**2))/L_tot
                                      
    sigma_s_lum = np.sqrt(sigma_s_lum_squared[0] + sigma_s_lum_squared[1] + sigma_s_lum_squared[2])
    
    return sigma_s_lum

The function *velocity dispersion 6* computes the velocity dispersion of binary stars, only, in the assumption that they are unresolved (the orbital motion is not ruled out); hence, it returns the velocity dispersion value *$\sigma_b$* (float type) in km/s.

In [ ]:
def velocity_dispersion_6(N_b, V_1, V_2):
    
    N_b_tot = 2*N_b
    
    V_b = np.zeros(shape=(N_b_tot,3))
    V_b[:len(V_1)] = V_1
    V_b[len(V_1):] = V_2
    
    V_b_mean = np.zeros(3)
    V_b_mean[0] = np.mean(V_b[:,0])
    V_b_mean[1] = np.mean(V_b[:,1])
    V_b_mean[2] = np.mean(V_b[:,2])
    
    sigma_b_squared = np.zeros(3)
    sigma_b_squared[0] = np.sum((V_b[:,0]-V_b_mean[0])**2)/N_b_tot
    sigma_b_squared[1] = np.sum((V_b[:,1]-V_b_mean[1])**2)/N_b_tot
    sigma_b_squared[2] = np.sum((V_b[:,2]-V_b_mean[2])**2)/N_b_tot
    
    sigma_b = np.sqrt(sigma_b_squared[0] + sigma_b_squared[1] + sigma_b_squared[2])
    
    return sigma_b

Parameters needed in the function *main program* below to realize a reference model of either a UFD or a dSph galaxy.

In [ ]:
# UFD reference model

G = 4.30145*10**(-3) # pc*(km/s)**2*(M_Sun)**(-1)
R = 50 # pc
M_tot = 5*10**4 # M_Sun
m_min = 0.1 # M_Sun
m_max = 50 # M_Sun
m_mean= 0.61 # M_Sun
a_au_min = 0.2 # AU
a_au_max = 100 # AU
e_min = 0
e_max = 1

# dSph reference model

G = 4.30145*10**(-3) # pc*(km/s)**2*(M_Sun)**(-1)
R = 3*10**3 # pc
M_tot = 10**7 # M_Sun
m_min = 0.1 # M_Sun
m_max = 50 # M_Sun
m_mean= 0.61 # M_Sun
a_au_min = 0.2 # AU
a_au_max = 100 # AU
e_min = 0
e_max = 1

The function *main program* initializes all the functions in the program once the free parameters *R*, *M$_{tot}$*, *m$_{min}$*, *m$_{max}$*, *m$_{mean}$*, *a$_{AU,min}$*, *a$_{AU,max}$*, *e$_{min}$* and *e$_{max}$* are specified. Thereby it applies a correction to stellar masses in order for the extracted value of the total mass of the system (*M*) to match the imposed one (*M$_{tot}$*), and computes the number of both single and binary stars, as well as the total number of stars in the system for a given binary fraction *f$_b$*.
In particular, the function *luminosity cut-off* is to be run only in the dSph case; if, on the other hand, a UFD is to be simulated, then the function *luminosity cut-off* must not be initialized by simply placing a # simbol before it.

In [27]:
def main_program(M_tot, f_b, m_mean):
    
    # Correction for the total mass of the system

    N_tot = round(M_tot/m_mean)
    
    m_tot = selection_masses(N_tot, m_min, m_max)
    M = np.sum(m_tot)
    correction = M_tot/M
    m_tot = m_tot*correction
    
    # Stellar numbers
    
    f_b /= 100
    N_b = round(float(f_b*N_tot))
    N_s = N_tot - 2*N_b
    N = N_s + N_b
    
    # Functions initialization
    
    m_s, m_1, m_2, m_cm, m, q = assign_masses(N_s, N_b, m_tot)
    
    N_remn, M_remn = find_remnants(m_tot)
    
    N_MS, N_RGB, N_WD = count_stars(m_tot)

    r, X = position(N, R)
    
    X_s = single_position(N_s, r, X)

    v, V = velocities(N, M_tot, R, r)
    
    v_s, V_s = single_velocities(N_s, v, V)

    Q = virial_ratio(G, m, X, v)
    
    a_au, a = semi_major_axis(N_b, a_au_min, a_au_max)
    
    P = period(m_1, a_au, q, G_au=39.478)
    
    e = eccentricity(N_b, e_min, e_max)
    
    X_cm, X_1, X_2 = binary_positions(N_s, N_b, m_1, m_2, r, X, a, e)
    
    V_cm, V_1, V_2 = binary_velocities(N_s, N_b, m_1, m_2, v, V, a, e)
    
    M_MS, M_RGB, log_L_MS, log_L_RGB = isochrone_parameters()
    
    L_s, L_1, L_2, T_s, T_1, T_2 = luminosity_temperature(M_MS, M_RGB, log_L_MS, log_L_RGB, log_T_MS, log_T_RGB, m_s, m_1, m_2)
    
    R_1, R_2 = stellar_radii(L_1, L_2, T_1, T_2)
    
    N_s, N_b, m_s, m_1, m_2, V_s, V_cm, V_1, V_2, L_s, L_1, L_2, R_1, R_2, a, e = luminosity_cut_off(m_s, m_1, m_2, V_s, V_cm, V_1, V_2, L_s, L_1, L_2, R_1, R_2, a, e)
    
    N_b_acc, N_b_rej, V_1_acc, V_2_acc, V_cm_acc, V_cm_rej, L_s, L_1_acc, L_2_acc, L_b_rej, a_au_acc, e_acc = RLOF(N_b, m_s, m_1, m_2, V_1, V_2, V_cm, L_1, L_2, R_1, R_2, a, a_au, e)
    
    histograms(V_s, V_1, V_2, a_au, a_au_acc, e, e_acc, _plot=True)

    sigma_tot = velocity_dispersion_1(N_s, N_b_acc, N_b_rej, V_s, V_cm_rej, V_1_acc, V_2_acc)
    
    sigma_sb = velocity_dispersion_2(N_s, N_b_acc, N_b_rej, V_s, V_cm_acc, V_cm_rej)
    
    sigma_s = velocity_dispersion_3(N_s, V_s)
    
    sigma_tot_lum = velocity_dispersion_4(N_s, N_b_acc, N_b_rej, V_s, V_1_acc, V_2_acc, V_cm_rej, L_s, L_1_acc, L_2_acc, L_b_rej)
    
    sigma_s_lum = velocity_dispersion_5(V_s, L_s)
    
    sigma_b = velocity_dispersion_6(N_b, V_1, V_2)
    
    # New binary fraction after the luminosity cut-off
    
    f_b_new = N_b/(N_s+2*N_b)
    
    return sigma_tot, sigma_sb, sigma_s, sigma_tot_lum, sigma_s_lum, sigma_b, N_MS, N_RGB, N_WD
    
    #return sigma_tot, sigma_sb, sigma_s, sigma_tot_lum, sigma_s_lum, sigma_b, N_MS, N_RGB, N_WD, f_b_new

Simulation launch commands and data gathering according to the binary fraction *f$_b$*. Each simulation is run a *N$_{run}$* times (the value of *N$_{run}$* is left to the user choice), so that mean values of the velocity dispersion and the number of MS stars, RGB stars and WDs can be computed.
Mean values of the velocity dispersion at varying binary fraction, together with the associated standard deviation (only for $\sigma_{tot}$ and $\sigma_{tot,lum}$), are stored in files named *sigma$_{100f_b}$*, whereas mean values of stellar numbers at varying binary fraction in files named *stars$_{100f_b}$*.
Output files can be printed with explicative headers of the stored quantities by writing *header=True*.

In [28]:
for f_b in [5,10,15,20,25,30,35,40]:
        
    sigma_tot, sigma_sb, sigma_s, sigma_tot_lum, sigma_s_lum, sigma_b = [],[],[],[],[],[]
    
    N_MS, N_RGB, N_WD = [],[],[]
    
    f_b_new = []
    
    print('# Doing fraction:', f_b)
    
    for i in range(N_run):

        sigma_1, sigma_2, sigma_3, sigma_4, sigma_5, sigma_6, N_1, N_2, N_3 = main_program(M_tot, f_b)
        #sigma_1, sigma_2, sigma_3, sigma_4, sigma_5, sigma_6, N_1, N_2, N_3, fraction = main_program(M_tot, f_b)
        
        sigma_tot.append(sigma_1)
        sigma_sb.append(sigma_2)
        sigma_s.append(sigma_3)
        sigma_tot_lum.append(sigma_4)
        sigma_s_lum.append(sigma_5)
        sigma_b.append(sigma_6)
        
        N_MS.append(N_1)
        N_RGB.append(N_2)
        N_WD.append(N_3)
        
        #f_b_new.append(fraction)
        
    # Velocity dispersion output files

    sigma_tot = np.asarray(sigma_tot)
    sigma_sb = np.asarray(sigma_sb)
    sigma_s = np.asarray(sigma_s)
    sigma_tot_lum = np.asarray(sigma_tot_lum)
    sigma_s_lum = np.asarray(sigma_s_lum)
    sigma_b = np.asarray(sigma_b)
    
    sigma_tot_mean = np.mean(sigma_tot)
    sigma_sb_mean = np.mean(sigma_sb)
    sigma_s_mean = np.mean(sigma_s)
    sigma_tot_lum_mean = np.mean(sigma_tot_lum)
    sigma_s_lum_mean = np.mean(sigma_s_lum)
    sigma_b_mean = np.mean(sigma_b)
    
    var_tot = np.sqrt(np.var(sigma_tot))
    var_tot_lum = np.sqrt(np.var(sigma_tot_lum))

    data_sigma = {'sigma_tot_mean': sigma_tot_mean, 'sigma_sb_mean': sigma_sb_mean, 'sigma_s_mean': sigma_s_mean, 
                  'sigma_tot_lum_mean': sigma_tot_lum_mean, 'sigma_s_lum_mean': sigma_s_lum_mean, 'sigma_b_mean':
                  sigma_b_mean, 'var_tot': var_tot, 'var_tot_lum': var_tot_lum}

    ser_sigma = pd.Series(data_sigma, ['sigma_tot_mean','sigma_sb_mean','sigma_s_mean','sigma_tot_lum_mean', 
                                       'sigma_s_lum_mean','sigma_b_mean','var_tot','var_tot_lum'])
    
    file_sigma = ser_sigma.to_csv(r'sigma_%i'%f_b, sep=' ', index=False, header=False)
    
    # Stars output files
    
    N_MS = np.asarray(N_MS)
    N_RGB = np.asarray(N_RGB)
    N_WD = np.asarray(N_WD)

    N_MS_mean = np.mean(N_MS)
    N_RGB_mean = np.mean(N_RGB)
    N_WD_mean = np.mean(N_WD)
    
    data_stars = {'N_MS_mean': N_MS_mean, 'N_RGB_mean': N_RGB_mean, 'N_WD_mean': N_WD_mean}
    
    ser_stars = pd.Series(data_stars, ['N_MS_mean','N_RGB_mean','N_WD_mean'])
    
    file_stars = ser_stars.to_csv(r'stars_%i'%f_b, sep='', index=False, header=False)
    
    # New binary fraction output file
    
    #f_b_new = np.asarray(f_b_new)
    #f_b_new_mean = np.mean(f_b_new)
    
    #data_fraction = {'f_b_new_mean': f_b_new_mean}
    #ser_fraction = pd.Series(data_fraction, ['f_b_new_mean'])
    #file_fraction = ser_fraction.to_csv(r'f_b_new_%i'%f_b, sep='', index=False, header=False)

# Doing fraction: 10
# Doing fraction: 20
# Doing fraction: 30
